In [1]:
%matplotlib inline
import sys, re
import numpy as np
import pandas as pd
import csv
import nltk
from bs4 import BeautifulSoup
import string
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
from matplotlib import pyplot as plt
pd.options.display.max_rows = 4000
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output, State
#import ipynb.fs
#from .full.app import app
#from .full.layouts import layout1, layout2, layout3
from jupyter_dash import JupyterDash
import dash_auth
import plotly
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from jitcache import Cache
import dash_table as dt
import dash_bootstrap_components as dbc
import plotly.graph_objs as go
from dash.exceptions import PreventUpdate
from pandas.tseries.offsets import *
import flask
import uuid
import datetime

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/visualboardwalk/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
electronic_data = pd.read_csv(r"/Volumes/GoogleDrive/Shared drives/MediSprout/data/cogent_flow_data/electronic_data.csv")
electronic_data.head(10)

,Date,Time,CC,Description,Price,Qty,Watch,Impl. Vol,Type,Strategy,Sub Type,Side,Product,RFQ,Add Deal,Undrl. Bid,Undrl. Offer,Δ,Category
0,8/17/2020,18:00:49,GE,Dec20 99.875/100.000 CS,0.5,50,NaN,NaN,Trade,CS,Trade,NaN,Eurodollar Option,NaN,NaN,99.715,99.720,NaN,Eurodollar and SOFR Options
1,8/18/2020,16:56:04,GE,Dec21 99.500 Put,4.0,499,NaN,29.06,Trade,Put,Trade,NaN,Eurodollar Option,NaN,NaN,99.765,99.770,-0.21,Eurodollar and SOFR Options
2,8/19/2020,16:48:23,GE2,Oct20 99.750/99.625/99.500 PTREE vs Eurodollar...,2.5,131,NaN,36.65,Trade,PTREE,Trade,NaN,Eurodollar Option 2-Year MC,NaN,NaN,99.720,99.725,-0.05,Eurodollar and SOFR Options
3,8/20/2020,16:48:10,GE2,Oct20 99.750/99.625/99.500 PTREE vs Eurodollar...,2.5,69,NaN,36.65,Trade,PTREE,Trade,NaN,Eurodollar Option 2-Year MC,NaN,NaN,99.720,99.725,-0.05,Eurodollar and SOFR Options
4,8/21/2020,16:47:53,GE2,Oct20 99.750/99.625/99.500 PTREE vs Eurodollar...,2.5,50,NaN,36.65,Trade,PTREE,Trade,NaN,Eurodollar Option 2-Year MC,NaN,NaN,99.720,99.725,-0.05,Eurodollar and SOFR Options
5,8/22/2020,16:45:20,GE2,Oct20 99.750/99.625/99.500 PTREE vs Eurodollar...,2.5,250,NaN,36.65,Trade,PTREE,Trade,NaN,Eurodollar Option 2-Year MC,NaN,NaN,99.720,99.725,-0.05,Eurodollar and SOFR Options
6,8/23/2020,16:32:38,GE,Jun21 99.500/99.750/100.000 CFLY,16.0,100,NaN,NaN,Trade,CFLY,Trade,NaN,Eurodollar Option,NaN,NaN,99.800,99.805,NaN,Eurodollar and SOFR Options
7,8/24/2020,16:19:20,GE,Sep20 98.7500 Call,101.5,100,NaN,NaN,Trade,Call,Trade,NaN,Eurodollar Option,NaN,NaN,99.765,99.770,NaN,Eurodollar and SOFR Options
8,8/25/2020,16:17:18,GE2,Sep20 99.625 Put vs Eurodollar Future Sep22 99...,1.0,"2,000",NaN,NaN,Cross,Put,Cross,NaN,Eurodollar Option 2-Year MC,NaN,NaN,NaN,NaN,NaN,Eurodollar and SOFR Options
9,8/26/2020,16:17:18,GE2,Sep20 99.625 Put vs Eurodollar Future Sep22 99...,1.0,"2,000",NaN,44.57,Trade,Put,Trade,NaN,Eurodollar Option 2-Year MC,NaN,NaN,99.755,99.760,-0.06,Eurodollar and SOFR Options


In [3]:
price_sorted = electronic_data.groupby('Price').sum().sort_values('Price', ascending=False).reset_index()
price_sorted

,Price,Watch,Impl. Vol,Side,RFQ,Add Deal,Undrl. Bid,Undrl. Offer,Δ
0,101.50,0.0,0.00,0.0,0.0,0.0,99.765,99.7700,0.00
1,59.00,0.0,52.98,0.0,0.0,0.0,99.710,99.7150,0.97
2,45.50,0.0,0.00,0.0,0.0,0.0,99.785,99.7900,0.00
3,22.00,0.0,0.00,0.0,0.0,0.0,99.770,99.7750,0.00
4,17.00,0.0,60.30,0.0,0.0,0.0,299.385,299.4000,0.54
5,16.00,0.0,0.00,0.0,0.0,0.0,99.800,99.8050,0.00
6,11.50,0.0,196.58,0.0,0.0,0.0,199.190,199.2050,1.10
7,11.00,0.0,0.00,0.0,0.0,0.0,99.725,99.7300,0.00
8,10.00,0.0,94.36,0.0,0.0,0.0,199.280,199.2900,0.22
9,9.50,0.0,32.89,0.0,0.0,0.0,199.565,199.5750,-0.63


In [4]:
quantity_sorted = electronic_data.groupby('Qty').sum().sort_values('Qty', ascending=False).reset_index()
quantity_sorted

,Qty,Price,Watch,Impl. Vol,Side,RFQ,Add Deal,Undrl. Bid,Undrl. Offer,Δ
0,99,3.50,0.0,15.08,0.0,0.0,0.0,99.7625,99.7650,0.26
1,98,3.50,0.0,15.08,0.0,0.0,0.0,99.7625,99.7650,0.26
2,944,3.00,0.0,0.00,0.0,0.0,0.0,99.7700,99.7750,0.00
3,80,1.25,0.0,22.53,0.0,0.0,0.0,99.7150,99.7200,-0.20
4,69,2.50,0.0,36.65,0.0,0.0,0.0,99.7200,99.7250,-0.05
5,67,18.00,0.0,20.10,0.0,0.0,0.0,199.5650,199.5750,0.18
6,65,3.50,0.0,14.74,0.0,0.0,0.0,99.7650,99.7675,0.31
7,63,3.50,0.0,14.74,0.0,0.0,0.0,99.7650,99.7675,0.31
8,600,1.00,0.0,26.24,0.0,0.0,0.0,99.7950,99.8000,-0.25
9,542,3.00,0.0,0.00,0.0,0.0,0.0,99.7700,99.7750,0.00


In [5]:
vol_sorted = electronic_data.groupby('Impl. Vol').sum().sort_values('Impl. Vol', ascending=False).reset_index()
vol_sorted

,Impl. Vol,Price,Watch,Side,RFQ,Add Deal,Undrl. Bid,Undrl. Offer,Δ
0,98.29,23.00,0.0,0.0,0.0,0.0,199.190,199.2050,1.10
1,82.91,0.25,0.0,0.0,0.0,0.0,99.795,99.8000,0.02
2,72.16,1.00,0.0,0.0,0.0,0.0,99.725,99.7300,0.04
3,60.67,8.00,0.0,0.0,0.0,0.0,199.450,199.4600,-0.18
4,60.22,3.00,0.0,0.0,0.0,0.0,199.290,199.3000,-0.36
5,52.98,59.00,0.0,0.0,0.0,0.0,99.710,99.7150,0.97
6,47.18,20.00,0.0,0.0,0.0,0.0,199.280,199.2900,0.22
7,44.57,1.00,0.0,0.0,0.0,0.0,99.755,99.7600,-0.06
8,36.65,10.00,0.0,0.0,0.0,0.0,398.880,398.9000,-0.20
9,35.70,3.00,0.0,0.0,0.0,0.0,99.765,99.7700,-0.13


In [6]:
electronic_data['Strategy'].value_counts()

STRD         30
Put          17
PCONDR        6
PTREE         5
Call          4
Custom        4
STNGL         4
PDOUBLE       3
CS 1x2        3
PS 1x4        3
CFLY          2
PS 2x3        2
PS            1
RR            1
CS            1
STRD SPRD     1
CCONDR        1
PS 1x3        1
Name: Strategy, dtype: int64

In [7]:
electronic_data.groupby('Strategy')['Date'].count().sort_values(ascending=False)

Strategy
STRD         30
Put          17
PCONDR        6
PTREE         5
STNGL         4
Custom        4
Call          4
CS 1x2        3
PS 1x4        3
PDOUBLE       3
CFLY          2
PS 2x3        2
STRD SPRD     1
CS            1
PS            1
PS 1x3        1
RR            1
CCONDR        1
Name: Date, dtype: int64

In [8]:
electronic_data["Date"] = pd.to_datetime(electronic_data["Date"])
electronic_data["Date"]

0    2020-08-17
1    2020-08-18
2    2020-08-19
3    2020-08-20
4    2020-08-21
5    2020-08-22
6    2020-08-23
7    2020-08-24
8    2020-08-25
9    2020-08-26
10   2020-08-27
11   2020-08-28
12   2020-08-29
13   2020-08-30
14   2020-08-31
15   2020-09-01
16   2020-09-02
17   2020-09-03
18   2020-09-04
19   2020-09-05
20   2020-09-06
21   2020-09-07
22   2020-09-08
23   2020-09-09
24   2020-09-10
25   2020-09-11
26   2020-09-12
27   2020-09-13
28   2020-09-14
29   2020-09-15
30   2020-09-16
31   2020-09-17
32   2020-09-18
33   2020-09-19
34   2020-09-20
35   2020-09-21
36   2020-09-22
37   2020-09-23
38   2020-09-24
39   2020-09-25
40   2020-09-26
41   2020-09-27
42   2020-09-28
43   2020-09-29
44   2020-09-30
45   2020-10-01
46   2020-10-02
47   2020-10-03
48   2020-10-04
49   2020-10-05
50   2020-10-06
51   2020-10-07
52   2020-10-08
53   2020-10-09
54   2020-10-10
55   2020-10-11
56   2020-10-12
57   2020-10-13
58   2020-10-14
59   2020-10-15
60   2020-10-16
61   2020-10-17
62   202

SINGLE PAGE TEMPLATE

In [67]:
df = electronic_data
df = df.replace(',','', regex=True)
df['Qty'] = df['Qty'].astype('float64')

app = JupyterDash(__name__, external_stylesheets=[
        'https://stackpath.bootstrapcdn.com/bootswatch/4.5.2/solar/bootstrap.min.css'])

def get_str_dtype(df, col):
    """Return dtype of col in df"""
    dtypes = ['object', 'float64', 'datetime64']
    for d in dtypes:
        if d in str(df.dtypes.loc[col]).lower():
            return d

app.layout = html.Div([html.Div(className="section",
    children=[
        dbc.Button(
            id="enter-button",
            className="button is-large is-outlined",
            children=["Export JSON"]
        ),
        html.Div(
            id="download-area",
            className="block",
            children=[]
        )
    ]),
    dt.DataTable(
        id='datatable-interactivity',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.loc[:, df.columns != 'Description']],
        data=df.to_dict('rows'),
        filter_action="native",
        sort_action="native",
        sort_mode="multi",
        selected_columns=[],
        selected_rows=[],
        page_action="native",
        page_current= 0,
        page_size= 15,
        export_format="csv"
    ),
        html.Div([
            dcc.DatePickerRange(id='date_filter',
                                clearable=True,
                                with_portal=True,
                                initial_visible_month= datetime.datetime(2020, 8, 1),
                               )], id='container_date_filter', ),
        html.Div([
            html.Label('Strategy'),
            dcc.Dropdown(id='dropdown_d1', options=[{'label':x, 'value':x} for x in df.sort_values('Strategy')['Strategy'].unique()], 
                         value=None, multi=True),
            html.Label('Product'),
                       dcc.Dropdown(id='dropdown_d2', options=[{'label': i, 'value': i} for i in df.sort_values('Strategy')['Product'].unique()],
                                    value=None, multi=True),
            html.Label('Type'),
            dcc.Dropdown(id='dropdown_d3', options=[{'label':x, 'value':x} for x in df.sort_values('Product')['Type'].unique()], 
                     value=None, multi=True), 
        ], style={'display': 'inline-block', 'width': '30%', 'margin-left': '7%'}),
            html.Div([
            html.Label('Price Slider'),
            html.Div(children=dcc.RangeSlider(id='price_slider',
                                              updatemode='drag',
                                              value = [df["Price"].min(), df["Price"].max()],
                                              allowCross = False
                                             )),
            html.Div(children=html.Div(id='price_slider_vals')),
        ], id='container_price_slider', 
                style={'display': 'inline-block', 'width': '30%', 'margin-left': '45%'} ),
            html.Div([
            html.Label('Qty Slider'),
            html.Div(children=dcc.RangeSlider(id='qty_slider',
                                              updatemode='drag',
                                              value = [df["Qty"].min(), df["Qty"].max()],
                                              allowCross = False
                                             )),
            html.Div(children=html.Div(id='qty_slider_vals'), ),
        ], id='container_qty_slider', 
                style={'display': 'inline-block', 'width': '30%', 'margin-left': '45%'} ),
            html.Div([
            html.Label('Impl. Vol Slider'),
            html.Div(children=dcc.RangeSlider(id='vol_slider',
                                              updatemode='drag',
                                              value = [df["Impl. Vol"].min(), df["Impl. Vol"].max()],
                                              allowCross = False
                                             )),
            html.Div(children=html.Div(id='vol_slider_vals'), ),
        ], id='container_vol_slider', 
                style={'display': 'inline-block', 'width': '30%', 'margin-left': '45%'} ),
            html.Div([html.Div(className="section2",
    children=[
        dbc.Button("Filter", color="primary", id="filter_button")
    ])
                     ]),
    html.Div(id='datatable-interactivity-container'),
    html.Div(
)
            ])
#JSON EXPORT
def build_download_button(uri):
    """Generates a download button for the resource"""
    button = html.Form(
        action=uri,
        method="get",
        children=[
            html.Button(
                className="button",
                type="submit",
                children=["Export JSON"]
            )
        ]
    )
    #return button

    
#BUILD PRICE SLIDER
@app.callback(Output('price_slider_vals', 'children'),
              [Input('price_slider', 'value')])
def build_price_slider(numbers):
    if numbers is None:
        raise PreventUpdate
    return '$' + ' to $'.join([str(numbers[0]), str(numbers[-1])])


#BUILD QTY SLIDER
@app.callback(Output('qty_slider_vals', 'children'),
              [Input('qty_slider', 'value')])
def build_qty_slider(numbers):
    if numbers is None:
        raise PreventUpdate
    return '' + ' to '.join([str(numbers[0]), str(numbers[-1])])

#BUILD IMPL. VOL SLIDER
@app.callback(Output('vol_slider_vals', 'children'),
              [Input('vol_slider', 'value')])
def build_qty_slider(numbers):
    if numbers is None:
        raise PreventUpdate
    return '' + ' to '.join([str(numbers[0]), str(numbers[-1])])

#FILTER TABLE
# FILTER DATA USING DATE PICKER / DROPDOWN / SLIDER RANGES

@app.callback(Output('datatable-interactivity', 'data'),
                  Input("filter_button", "n_clicks"),
                  State('price_slider', 'value'),
                   State('qty_slider', 'value'),
                   State('vol_slider', 'value'),
                   State('date_filter', 'start_date'),
                   State('date_filter', 'end_date'),
                   State('dropdown_d1', 'value'),
                   State('dropdown_d2', 'value'),
                   State('dropdown_d3', 'value'))

def filter_table(n_clicks, num1, num2, num3, start_date, end_date, d1, d2, d3):
    
    filter_df = df
    ## date filter
    if start_date and end_date:
        filter_df = filter_df.query("Date >= @start_date and Date <= @end_date")
    elif start_date:
        filter_df = filter_df.query("Date >= @start_date")
    elif end_date:
        filter_df = filter_df.query("Date <= @end_date")
    else:
        pass        
 
    
    ## multi-select dropdowns
    if d1:
        filter_df = filter_df.query("Strategy in @d1")
    if d2:
        filter_df = filter_df.query("Product in @d2")
    if d3:
        filter_df = filter_df.query("Type in @d3")
        
    ## slider ranges
    if num1:
        ## price 
        filter_df = filter_df[filter_df["Price"].between(num1[0], num1[-1])]
    if num2:
        # quantity
        filter_df = filter_df[filter_df["Qty"].between(num2[0], num2[-1])]
    if num3:
        # implied vol
        filter_df = filter_df[filter_df["Impl. Vol"].between(num3[0], num3[-1])]
    
    return filter_df.to_dict("rows")
    print(filter_df)


#UPDATE GRAPHS
@app.callback(
    Output('datatable-interactivity-container', "children"),
    Input('datatable-interactivity', "derived_virtual_data"),
    Input('datatable-interactivity', "derived_virtual_selected_rows"),
    )
def update_graphs(rows, derived_virtual_selected_rows):
    # When the table is first rendered, `derived_virtual_data` and
    # `derived_virtual_selected_rows` will be `None`. This is due to an
    # idiosyncrasy in Dash (unsupplied properties are always None and Dash
    # calls the dependent callbacks when the component is first rendered).
    # So, if `rows` is `None`, then the component was just rendered
    # and its value will be the same as the component's dataframe.
    # Instead of setting `None` in here, you could also set
    # `derived_virtual_data=df.to_rows('dict')` when you initialize
    # the component.
    if derived_virtual_selected_rows is None:
        derived_virtual_selected_rows = []

    dff = df if rows is None else pd.DataFrame(rows)

    colors = ['#7FDBFF' if i in derived_virtual_selected_rows else '#0074D9'
              for i in range(len(dff))]

    return [
        dcc.Graph(
            id=column,
            figure={
                "data": [
                    {
                        "x": dff["Strategy"],
                        "y": dff[column],
                        "type": "bar",
                        "marker": {"color": colors},
                    }
                ],
                "layout": {
                    "xaxis": {"automargin": True},
                    "yaxis": {
                        "automargin": True,
                        "title": {"text": column}
                    },
                    "height": 250,
                    "margin": {"t": 10, "l": 10, "r": 10},
                },
            },
        )
        # check if column exists - user may have deleted it
        # If `column.deletable=False`, then you don't
        # need to do this check.
        for column in ["Price", "Qty", "Impl. Vol"] if column in dff
    ]


@app.callback(
    Output("download-area", "children"),
    [
        Input("enter-button", "n_clicks")
    ])

def show_download_button(n_clicks):
    # turn text area content into file
    filename = f"{uuid.uuid1()}.json"
    path = r"/Users/visualboardwalk/Documents/cogent_export"
    with open(path, "w") as file:
        file.write(pd.DataFrame.from_dict(filter_table()).to_json(orient="records", lines=True))
    uri = path
    return [build_download_button(uri)]


@app.server.route('/downloadable/<path:path>')
def serve_static(path):
    root_dir = os.getcwd()
    return flask.send_from_directory(
        os.path.join(root_dir, 'downloadable'), path
    )


if __name__ == '__main__':
    app.run_server(port = 8090, dev_tools_ui=True, debug=True,
              dev_tools_hot_reload =True, threaded=True)

Dash app running on http://127.0.0.1:8090/


In [64]:
filter_table()

KeyError: 'outputs_list'

In [10]:
df.dtypes

Date            datetime64[ns]
Time                    object
CC                      object
Description             object
Price                  float64
Qty                    float64
Watch                  float64
Impl. Vol              float64
Type                    object
Strategy                object
Sub Type                object
Side                   float64
Product                 object
RFQ                    float64
Add Deal               float64
Undrl. Bid             float64
Undrl. Offer           float64
Δ                      float64
Category                object
dtype: object

In [11]:
df['Type'].unique()

array(['Trade', 'Cross'], dtype=object)